In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
import os
from pathlib import Path

path_cwd=Path.cwd()
path_input=str(path_cwd)+'/Data_input/'
path_output=str(path_cwd)+'/Data_output/'

In [4]:
#read
df = pd.read_csv(path_input+'ENVIRONMENTAL_tidy.csv', index_col='key_0',parse_dates=['key_0'])

In [5]:
# In-Out
Precipitation_2021= df.filter(items=['Rain(mm)']).loc['2021-05-01 00:30:00':'2021-09-15 00:30:00']
PET_2021= df.filter(items=['PET(mm/h)']).loc['2021-05-01 00:30:00':'2021-09-15 00:30:00']
Precipitation_2022= df.filter(items=['Rain(mm)']).loc['2022-05-01 00:30:00':'2022-09-15 00:30:00']
PET_2022= df.filter(items=['PET(mm/h)']).loc['2022-05-01 00:30:00':'2022-09-15 00:30:00']

In [6]:
# General simulation parameters
Nt = len(Precipitation_2021) # total half hours 

In [7]:
# Reservoirs-Stocks
deep_fracture = [None]*Nt #closed L
surface_fracture = [None]*Nt #open L
quickflow= [None]*Nt #open L
tree = [None]*Nt #water sink but also a stock L

# Initial conditions
quickflow [0] = 0.001 # L
deep_fracture[0] = 0.3 # L
surface_fracture[0] = 0.1 # L 
tree[0] = 0.1 #L

In [12]:
# Flows- Record all flow channels
q1 = [None]*Nt # precipitation -> quickflow
q2 = [None]*Nt # precipitation -> soil
q3 = [None]*Nt # soil -> fracture 
q3_1 = [None]*Nt # fracture -> soil 
q3_2 = [None]*Nt #soil -> quickflow 
q4 = [None]*Nt # fracture -> plant
q4_1 = [None]*Nt # fracture -> air
q5 = [None]*Nt # soil -> plant
q5_1 = [None]*Nt # soil -> air
q6 = [None]*Nt # plant -> air
sapflow = [None]*Nt


# Drivers and constrains 
#pet = [None]*Nt
X=0.03 #concentrated fraction that is lost from P to seepage almost immediately 
sf_max= 1 #maximum capacity for surface fracture L
df_max= 3 #maximum capacity for deep fracture L
tree_max= 2 #maximum capacity for tree L


In [20]:
precipitation=((Precipitation_2021['Rain(mm)'].values)*50856*1e-6).tolist() #50,856 mm2 area of rain gauge. 1e-6 mm3 to dm3 
pet=PET_2021['PET(mm/h)'].tolist() #calculated every 30 min and assumed to be over 1m2 -> mm~L
pet_n=np.zeros(len(pet)).tolist()
dates=Precipitation_2021.index.tolist()

In [43]:
for n in range(Nt):
    # Precipitation
    # Law: fraction lost to quickflow and fraction into soil 
    if precipitation[n] > 0:
        if surface_fracture[n] < sf_max:
            q1[n] = precipitation[n]*X # quickflow loss 
            q2[n] = precipitation[n]*(1-X) #infiltration into soil 
            surface_fracture[n] += q2[n]
            quickflow[n] += q1[n]
        else: 
            q1[n] = 0 #precipitation[n]*X #quickflow, the rest is assumed to be lost by overflow out of the system 
            q2 [n] = 0
            surface_fracture[n] += q2[n]
            quickflow[n] += q1[n]
    else: #not necessarily 
        q1[n] = 0
        q2[n] = 0

    # Soil-fracture mechanism
    # Law: water flows from soil to fracture 
    #      flow rate is proportional to reservoir volume
    if surface_fracture[n]>0:
        if (surface_fracture[n] > deep_fracture[n])&(deep_fracture[n]<df_max): 
            q3[n] = 0.5*(surface_fracture[n]/sf_max)
            deep_fracture[n] += q3[n]
            surface_fracture[n] -= q3[n]
        else: 
            q3[n] = 0
            deep_fracture[n] += q3[n] #may be redundant
            surface_fracture[n] -= q3[n]
    else:
        q3[n] = 0

    #Fracture-soil mechanism 
    # Law: water flows from fracture to soil at a slower rate 
    if deep_fracture[n] > 0:
        if (surface_fracture[n] < deep_fracture[n])&(surface_fracture[n]<sf_max):
            q3_1[n] = 0.001*(deep_fracture[n]/df_max)
            surface_fracture[n] += q3_1[n]
            deep_fracture[n] -= q3_1[n]
        else: 
            q3_1[n] = 0
    else:
        q3_1[n] = 0

    # Soil to quickflow (seepage)
    # Law: by changing proportionality constant we change how connected-open are the surface fractures to open ones 
    if surface_fracture[n] > 0: #if open it'll always flow 
        q3_2[n] = 0.5*(surface_fracture[n]/sf_max)
        quickflow[n] += q3_2[n]
        surface_fracture[n] -= q3_2[n]
    else:
        q3_2[n] = 0

    #Fracture-plant mechanism
    if deep_fracture[n] >= pet[n]*(deep_fracture[n]/df_max):
        q4[n] = pet[n]*(deep_fracture[n]/df_max)
        if tree[n] < tree_max: #storage
            tree[n] += 0.5*q4[n]
            q4_1[n] = 0.5*q4[n] #ET
            deep_fracture[n] -= q4[n]   
        else: 
            q4_1[n] = q4[n] #ET
            deep_fracture[n] -= q4[n]   
    else:
        q4[n] = 0

    # Soil-plant mechanism
    if surface_fracture[n] > 0:
        q5[n] = pet[n]*(surface_fracture[n]/sf_max)
        if tree[n]<tree_max: #storage
            tree[n] += 0.5*q5[n]
            q5_1[n]= 0.5*q5[n] #ET
            surface_fracture[n] -= q5[n]
        else:
            q5_1[n]= q5[n]
            surface_fracture[n] -= q5[n]
    else:
        q5[n] = 0


    # Plant sap-flow and transpiration
    # Law: transpiration = q6
    if tree[n] <= tree_max:
        q6[n] = pet[n]*(tree[n]/tree_max)
        tree[n] -= q6[n]
    else:
        q6[n] = 0

    pet_n[n]=(pet[n]-min(pet))/(max(pet)-min(pet))
    sapflow[n] = ((1/pet[n])*(q4_1[n] + q5_1[n] + q6[n]))*pet_n[n]


    # Copy computed results to future step
    if n < (Nt-1):
        surface_fracture[n+1] = surface_fracture[n]
        deep_fracture[n+1] = deep_fracture[n]
        quickflow [n+1] = quickflow[n]
        tree[n+1] = tree[n]
        #sapflow[n+1]=sapflow[n]
    


I need a rule that if theres water in the reservoir but pet is above certain treshold you will move less water 


In [44]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=dates, y=sapflow,
                    mode='lines',
                    name='sapflow'))
fig.add_trace(go.Scatter(x=dates, y=tree,
                    mode='lines',
                    name='tree'))
fig.add_trace(go.Scatter(x=dates, y=surface_fracture,
                    mode='lines',
                    name='surface'))
fig.add_trace(go.Scatter(x=dates, y=deep_fracture,
                    mode='lines',
                    name='deep'))
fig.add_trace(go.Scatter(x=dates, y=precipitation,
                    mode='lines',
                    name='precipitation'))
fig.add_trace(go.Scatter(x=dates, y=pet,
                    mode='lines',
                    name='PET'))

fig.show()